In [7]:
import matplotlib.pyplot as plt
import pickle
import numpy as np
import os
import pandas as pd
#from scipy.io import loadmat
import h5py
import numpy as np
import copy as cp
import pylab as pl
import math as mt

from maze_graph import *

file = pd.read_excel("D:\YSY\mice_maze_metadata_time_correction.xls", sheet_name = "training_recording_new")
totalpath = "D:\YSY"

def GetCellregFile(path):
    if os.path.exists(path):
        # Read cellRegistered.mat
        f = h5py.File(path,'r')
        ms_reg = f['cell_registered_struct']
        index_map = np.array(ms_reg['cell_to_index_map'])
        return index_map
    else:
        print("The path '"+str(path)+"' is not exist!")
        return []

In [8]:
# index map sorting, 
def NumOfDaysRecorded(index_map):
    RecordVec = []
    for i in range(np.shape(index_map)[1]):
        sum = 0
        for j in range(np.shape(index_map)[0]):
            if index_map[j][i] != 0:
                sum = sum + 1
        RecordVec = RecordVec + [sum]
    return RecordVec

def partition(index_map, RecordVec, low, high): 
    i = low - 1        
    pivot = RecordVec[high]       
    for j in range(low, high): 
        
        if   RecordVec[j] > pivot:           
            i = i+1
            RecordVec[i], RecordVec[j] = RecordVec[j], RecordVec[i]
            c = cp.deepcopy(index_map[:,i])
            index_map[:, i] = cp.deepcopy(index_map[:, j])
            index_map[:, j] = cp.deepcopy(c)      
            
    RecordVec[i+1], RecordVec[high] = RecordVec[high], RecordVec[i+1]
    c = cp.deepcopy(index_map[:,i+1])
    index_map[:, i+1] = cp.deepcopy(index_map[:, high])
    index_map[:, high] = cp.deepcopy(c)
    return ( i+1 ) 
    
def quickSort(index_map, RecordVec, low, high): 
    if low < high:   
        pi = partition(index_map, RecordVec, low, high)
        quickSort(index_map, RecordVec, low, pi-1)
        quickSort(index_map, RecordVec, pi+1, high)

def sort(index_map):
    RecordVec = NumOfDaysRecorded(index_map = index_map)
    print("RecordVec:")
    print(np.shape(RecordVec))
    print(RecordVec)
    quickSort(index_map, RecordVec, 0, len(RecordVec)-1)

In [9]:
def plot_rate_map(incor_rate_map,maze_type,ax=None,title=None,title_color = 'black',fontsize = 12):
    # plot the rate map in one day
    ma = ax.imshow(incor_rate_map,interpolation='none',origin = "lower",alpha = 1,aspect = 'auto')
    ax.set_title(title, fontsize = fontsize, color = title_color)
    ax.set_ylabel("Number of Neuron", fontsize = fontsize)
    am = plt.colorbar(ma,ax = ax, fraction = 0.08)
    am.set_label("Firing Rate")

def GenerateFigureSize(Size = 4):
    single_width = Size * 0.7
    single_highth = Size * 0.5
    fontsize = 3 * Size - 1
    linewidth = 0.25 * Size
    return single_width,single_highth,fontsize,linewidth
    
# draw maze outline
def isNorthBorder(mazeID):
    if mazeID/12 > 11:
        return 1
    else:
        return 0

def isEastBorder(mazeID):
    if mazeID % 12 == 0:
        return 1
    else:
        return 0

def isWestBorder(mazeID):
    if mazeID % 12 == 1:
        return 1
    else:
        return 0

def isSouthBorder(mazeID):
    if mazeID <= 12:
        return 1
    else:
        return 0

def OrdiToID(x,y):
    return y*12 + x + 1
    
def drawMazeOnRatemap(maze_type,axes, color = 'white',linewidth = 1):
    graph = maze1_graph if maze_type == 1 else maze2_graph
    for i in range(1,145):
        x = (i-1) % 12
        y = int((i-1) / 12)
        surr = graph[i]
        border = [0,0,0,0] # E,S,W,N
        if isEastBorder(i):
            axes.plot([x+0.5, x+0.5],[y+0.5, y-0.5],'-',color = color,linewidth = linewidth)
            border[0] = 1
        if isSouthBorder(i):
            axes.plot([x-0.5, x+0.5],[y-0.5, y-0.5],'-',color = color,linewidth = linewidth)
            border[1] = 1
        if isWestBorder(i):
            axes.plot([x-0.5, x-0.5],[y-0.5, y+0.5],'-',color = color,linewidth = linewidth)
            border[2] = 1
        if isNorthBorder(i):
            axes.plot([x-0.5, x+0.5],[y+0.5, y+0.5],'-',color = color,linewidth = linewidth)
            border[3] = 1
        for k in range(4):
            if border[0] != 1 and OrdiToID(x+1,y) not in surr:
                axes.plot([x+0.5, x+0.5],[y-0.5, y+0.5],'-',color = color,linewidth = linewidth)
            if border[1] != 1 and OrdiToID(x,y-1) not in surr:
                axes.plot([x-0.5, x+0.5],[y-0.5, y-0.5],'-',color = color,linewidth = linewidth)
            if border[2] != 1 and OrdiToID(x-1,y) not in surr:
                axes.plot([x-0.5, x-0.5],[y-0.5, y+0.5],'-',color = color,linewidth = linewidth)
            if border[3] != 1 and OrdiToID(x,y+1) not in surr:
                axes.plot([x-0.5, x+0.5],[y+0.5, y+0.5],'-',color = color,linewidth = linewidth)

In [10]:
def GenerateRatemap(miceID = 0, maze_type = 1):
    ind = np.where((file['number'] == miceID) & (file['maze_type'] == maze_type))[0]
    f = []
    for i in ind:
        FR_path = os.path.join(totalpath,str(miceID),str(file['date'][i]),"rate_map_all.pkl")
        if os.path.exists(FR_path):      
            with open(FR_path, 'rb') as handle:
                is_placecell_isi, SI_all, rate_map_all = pickle.load(handle)
                f.append([is_placecell_isi, SI_all, rate_map_all])
        else:
            f.append([])  # if rate_map_all.pkl file is lost, append an empty list into f list, and you can check if len(f[i]) == 0 to identify whether a day's data is lost
    return f

In [13]:
# Clean the NaN value
def Clear_NaN(rate_map_all):
    for j in range(len(rate_map_all)):
        for k in range(len(rate_map_all[0])):
            if mt.isnan(rate_map_all[j][k]):
                rate_map_all[j][k] = 0.0
    return rate_map_all

# plot days align
def plot_days_aligned_ratemap(maze_type = 1, index_map = None, miceID = 0,tit = None,filename = None):
    
    print("    Index_map:")
    print(np.shape(index_map))
    print(index_map)
    sort(index_map = index_map)
    print("    Sorted, Index_map:")
    print(np.shape(index_map))
    print(index_map)
    
    n_neuron = len(index_map[0])
    ind = np.where((file['maze_type'] == maze_type) & (file['number'] == miceID))[0]
    cols = len(ind)
    rows = n_neuron
    f = GenerateRatemap(miceID = miceID, maze_type = maze_type)
    
    if miceID == 10028 and maze_type == 1:
        Size = 3
    else:
        Size = 4
    
    if n_neuron >= 300:
        Size = 3
    
    single_width,single_highth,fontsize,linewidth = GenerateFigureSize(Size = Size)


    print(cols,rows)
    fig = plt.figure(figsize = (cols * single_width, rows * single_highth))
    for i in range(n_neuron):
        print("    neuronID:",i+1)
        for d in range(cols):
            if len(f[d]) == 0:   # check if rate_map_all.pkl file is lost
                continue
                
            if index_map[d,i] == 0:
                continue
            index = index_map[d,i] - 1
            rf = plt.subplot(rows, cols, cols*i + d + 1)
            rate_map_all = Clear_NaN(f[d][2])
            title = "%d %s (%.2f)" % (index, "day "+str(d+1), f[d][1][int(index)]) + tit
            if f[d][0][int(index)] == 1:
                plot_rate_map(np.reshape(rate_map_all[int(index),], [12,12]),2,rf,title=title,title_color = 'red',fontsize = fontsize)
                drawMazeOnRatemap(2,rf,linewidth = linewidth)
            else:
                plot_rate_map(np.reshape(rate_map_all[int(index),], [12,12]),2,rf,title=title, fontsize = fontsize)
                drawMazeOnRatemap(2,rf,linewidth = linewidth)
            rf.invert_yaxis()

    plt.tight_layout()
    print("    Save figure...")
    plt.savefig(os.path.join(totalpath,str(miceID),"Days_aligned","maze"+str(maze_type),'aligned_rate_map_'+filename+'.pdf'))
    plt.savefig(os.path.join(totalpath,"Days_Aligned_ratemap",'aligned_rate_map_'+filename+'.pdf'))
    plt.close()
    print("    Done.")
    
def Plot_all_mice():
    MiceID = np.unique(file['number'])
    for i in range(1,len(MiceID)):
        print("Now let's plot figure of mice "+str(MiceID[i]),'...')
        # when maze_type = 1
        
        path = os.path.join(totalpath,str(MiceID[i]),"Days_aligned","maze1","cellRegistered.mat")
        if os.path.exists(path):
            print("  maze1...")
            index_map = GetCellregFile(path)
            sort(index_map)
            sort(index_map)
            # plot_days_aligned_ratemap(index_map = index_map, maze_type = 1, miceID = MiceID[i], tit = ' maze1 '+str(MiceID[i]),filename = 'maze1_'+str(MiceID[i]))
        else:
            print("  maze1...")
            print("  The path '"+str(path)+"' is not exist!")
        print("  maze1 finished.")
        print("----------------------------------------------------------------------------------------------------------------------",end='\n\n')
        
        print("Now let's plot figure of mice "+str(MiceID[i]),'...')
        path = os.path.join(totalpath,str(MiceID[i]),"Days_aligned","maze2","cellRegistered.mat")
        if os.path.exists(path):
            print("  maze2...")
            index_map = GetCellregFile(path)
            sort(index_map)
            sort(index_map)
            # plot_days_aligned_ratemap(index_map = index_map, maze_type = 2, miceID = MiceID[i], tit = ' maze2 '+str(MiceID[i]),filename = 'maze2_'+str(MiceID[i]))
        else:
            print("  maze2...")
            print("  The path '"+str(path)+"' is not exist!") 
        print("  maze2 finished.")
        print("----------------------------------------------------------------------------------------------------------------------",end='\n\n')
        
        
Plot_all_mice()   

Now let's plot figure of mice 10021 ...
  maze1...
RecordVec:
(281,)
[6, 1, 1, 1, 3, 3, 3, 1, 2, 1, 4, 1, 1, 1, 1, 5, 1, 1, 1, 4, 2, 3, 2, 1, 1, 1, 1, 5, 1, 3, 1, 2, 3, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 1, 4, 2, 3, 4, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 3, 1, 1, 1, 2, 1, 3, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 5, 1, 1, 2, 1, 2, 1, 2, 1, 1, 1, 4, 2, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 2, 1, 4, 2, 2, 4, 3, 2, 7, 1, 3, 3, 2, 1, 7, 5, 3, 7, 3, 1, 5, 4, 2, 2, 2, 5, 2, 3, 1, 1, 4, 3, 2, 3, 2, 3, 3, 3, 2, 7, 3, 4, 3, 2, 1, 1, 2, 2, 1, 1, 1, 1]
RecordVec:
(281,)
[7, 7, 7, 7, 6, 5, 5, 5, 5, 5, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3,

  maze2...
RecordVec:
(104,)
[3, 2, 2, 1, 2, 6, 2, 3, 1, 7, 4, 5, 5, 2, 6, 6, 5, 6, 7, 5, 4, 4, 1, 5, 7, 1, 1, 2, 1, 2, 1, 3, 4, 5, 3, 5, 6, 4, 1, 2, 5, 3, 4, 4, 5, 1, 3, 4, 4, 2, 4, 4, 1, 3, 2, 1, 1, 1, 4, 1, 6, 1, 3, 1, 3, 1, 2, 1, 4, 4, 1, 3, 1, 2, 2, 3, 1, 2, 4, 1, 1, 1, 1, 2, 4, 2, 1, 1, 2, 6, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
RecordVec:
(104,)
[7, 7, 7, 6, 6, 6, 6, 6, 6, 6, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
  maze2 finished.
----------------------------------------------------------------------------------------------------------------------

Now let's plot figure of mice 10026 ...
  maze1...
RecordVec:
(280,)
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

  maze2 finished.
----------------------------------------------------------------------------------------------------------------------

Now let's plot figure of mice 10030 ...
  maze1...
RecordVec:
(488,)
[1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 3, 1, 3, 2, 4, 3, 1, 3, 1, 1, 3, 1, 1, 1, 3, 1, 1, 4, 3, 1, 2, 4, 1, 1, 3, 3, 3, 3, 3, 3, 3, 2, 1, 3, 1, 3, 3, 3, 1, 3, 1, 1, 1, 1, 3, 1, 3, 3, 3, 3, 2, 3, 2, 4, 1, 1, 1, 3, 1, 2, 4, 1, 3, 1, 1, 3, 3, 2, 3, 1, 1, 2, 1, 3, 2, 1, 2, 3, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2, 2, 3, 3, 2, 2, 2, 2, 2, 2, 1, 2, 1, 1, 2, 2, 1, 1, 1, 1, 2, 2, 2, 2, 1, 2, 1, 2, 2, 1, 1, 1, 1, 2, 2, 1, 1, 2, 2, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

RecordVec:
(183,)
[8, 7, 8, 1, 4, 6, 6, 1, 6, 6, 5, 7, 1, 1, 1, 1, 1, 1, 1, 1, 1, 7, 1, 7, 6, 7, 6, 8, 7, 7, 8, 7, 5, 1, 7, 2, 3, 1, 5, 5, 1, 3, 3, 5, 2, 5, 4, 6, 1, 7, 3, 6, 2, 7, 8, 6, 1, 6, 2, 4, 4, 2, 2, 1, 2, 4, 1, 6, 8, 6, 7, 3, 8, 3, 4, 6, 6, 6, 5, 3, 8, 4, 4, 3, 5, 6, 6, 2, 3, 3, 1, 2, 1, 4, 1, 1, 7, 1, 1, 4, 4, 5, 1, 1, 1, 1, 7, 1, 1, 1, 1, 1, 1, 1, 1, 1, 7, 1, 1, 1, 1, 1, 1, 3, 5, 3, 6, 2, 4, 6, 1, 3, 2, 2, 7, 1, 1, 1, 1, 6, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 6, 4, 4, 2, 2, 6, 1, 1, 6, 1, 3, 6, 5, 7, 2, 4, 4, 2, 1, 1, 4, 2, 2, 2, 2, 3, 1, 2, 1, 2, 1, 2, 1]
RecordVec:
(183,)
[8, 8, 8, 8, 8, 8, 8, 8, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,